In [1]:
! pip install pytube


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from pytube import YouTube
import cv2
import os

# Function to extract frames from a YouTube video with a specific frame rate
def extract_frames(video_url, output_folder, frame_rate):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Download the YouTube video
    yt = YouTube(video_url)
    stream = yt.streams.filter(file_extension='mp4').first()
    stream.download(output_folder)

    # Open the downloaded video file
    video_path = os.path.join(output_folder, stream.default_filename)
    cap = cv2.VideoCapture(video_path)

    # Extract frames at the specified frame rate
    frame_count = 0
    frame_skip = int(cap.get(cv2.CAP_PROP_FPS) / frame_rate)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        # Save frame as an image file
        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        # Skip frames based on frame_skip
        for _ in range(frame_skip - 1):
            cap.grab()

    cap.release()
    cv2.destroyAllWindows()
    print(f"Extracted frames at a rate of {frame_rate} frames per second.")

# Example usage
video_url = "https://www.youtube.com/watch?v=_pHZqBQYRQ4&t=277s"
output_folder = "C:\\Users\\hp\\Desktop\\Pivaga\\frames1"
frame_rate = 1  # Desired frame rate (frames per second)
extract_frames(video_url, output_folder, frame_rate)


Extracted frames at a rate of 1 frames per second.


In [ ]:
from pytube import YouTube
import cv2
import os

# Function to extract frames from a YouTube video with a specific frame rate
def extract_frames(video_url, output_folder, frame_rate):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Download the YouTube video
    yt = YouTube(video_url)
    stream = yt.streams.filter(file_extension='mp4').first()
    stream.download(output_folder)

    # Open the downloaded video file
    video_path = os.path.join(output_folder, stream.default_filename)
    cap = cv2.VideoCapture(video_path)

    # Extract frames at the specified frame rate
    frame_count = 0
    frame_skip = int(cap.get(cv2.CAP_PROP_FPS) / frame_rate)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        # Save frame as an image file
        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        # Skip frames based on frame_skip
        for _ in range(frame_skip - 1):
            cap.grab()

    cap.release()
    cv2.destroyAllWindows()
    print(f"Extracted frames at a rate of {frame_rate} frames per second.")

# Example usage
video_url = "https://www.youtube.com/watch?v=_pHZqBQYRQ4&t=277s"
# Change the output folder as per need, I worked on my own system and saved 
# on Desktop, , Ryzen 3
output_folder = "C:\\Users\\hp\\Desktop\\Pivaga\\Test"
frame_rate = 1  # Desired frame rate (frames per second)
extract_frames(video_url, output_folder, frame_rate)

In [ ]:
import os
import pandas as pd
import easyocr
from datetime import datetime

# Main directory path
# Result stores the cropped bounding boxes from
# the folder Test, you can modify according to your own directories
main_dir = r"C:\Users\hp\Downloads\CRAFT-pytorch\result"

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Initialize DataFrame to store data
data = pd.DataFrame(columns=['Text', 'Start Timestamp', 'End Timestamp'])

subfolders = [subfolder for subfolder in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, subfolder))]
#subfolders = os.listdir(main_dir)

# Sort the subfolder names based on their numeric values
# The results are saved under the folders with frame numbers
# for ex : res_frame_1 will contain the cropped bounding box 
# images for frame_1 and res_frame_2 will contain the cropped bounding box 
# images for frame_2 and so on
#
subfolders.sort(key=lambda x: int(x.split('_')[-1]))


# Iterate through subfolders
# This module is creating an excel sheet that stores identified shop names
for folder_name in subfolders:
    folder_path = os.path.join(main_dir, folder_name)
    if os.path.isdir(folder_path):
        #print("This folder path is being processed", folder_path)
        # Extract frame number from folder name
        frame_number = int(folder_name.split('_')[-1])
        start_timestamp = frame_number
        end_timestamp = frame_number+1
        
        # Iterate through images in subfolder
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            
            # Extract text using EasyOCR
            result = reader.readtext(img_path)
            extracted_text = ' '.join([res[1] for res in result])
            
            if len(extracted_text) != 0 :
                # Append data to DataFrame
                data = data.append({'Text': extracted_text,
                                    'Start Timestamp': start_timestamp,
                                    'End Timestamp': end_timestamp},
                                   ignore_index=True)

# Write DataFrame to Excel
output_file = r'C:\Users\hp\Downloads\CRAFT-pytorch\extracted_text.xlsx'
data.to_excel(output_file, index=False)

print(f"Extracted text saved to {output_file}")


In [ ]:
import pandas as pd

# Read the Excel file into a DataFrame
excel_file = r'C:\Users\hp\Downloads\CRAFT-pytorch\extracted_text.xlsx'
data = pd.read_excel(excel_file)

# Words to be excluded
excluded_words = ['the', 'and', 'ann', 'ad' 'or', 'floor', 'walking', 'tour', 'mall', 'of', 'in', 'at', 'th', 'te']

# Initialize a dictionary to store previously seen texts and their start timestamps
seen_texts = {}

# This module is to find shop names in created excel sheet and match it against other 
# shop names and timesstamp it accordingly. Since OCR accuracy is not very high, we are 
# ignoring minor mismatch of characters and taking the shop name as same
# Ex : Solly, Soily and Soly are counted as one shop name, Solly being the actually correct one
# Also we are removing common names like floor and the because they are used by vlogger for
# his own purpose
# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    text = row['Text']
    start_timestamp = row['Start Timestamp']
    end_timestamp = row['End Timestamp']
    
    # Check if the text matches any of the excluded words (case insensitive)
    if text.lower() in excluded_words:
        # Delete the current row
        data = data.drop(index)
    else:
        # Convert the current text to lowercase for case insensitive comparison
        text_lower = text.lower()
        
        # Check if the text matches any previously seen text based on your condition (case insensitive)
        matched_text = None
        for seen_text, seen_start_timestamp in seen_texts.items():
            seen_text_lower = seen_text.lower()
            if text_lower[0] == seen_text_lower[0] and text_lower[-1] == seen_text_lower[-1] and len(set(text_lower[1:-1]).intersection(set(seen_text_lower[1:-1]))) > 2:
                matched_text = seen_text
                break
        
        # If a match is found, update the end timestamp of the corresponding row
        if matched_text is not None:
            data.loc[data['Text'] == matched_text, 'End Timestamp'] = end_timestamp
            # Delete the current row
            data = data.drop(index)
        else:
            # Add the current text to the dictionary of seen texts
            seen_texts[text] = start_timestamp

# Write the updated DataFrame to Excel
output_file = r'C:\Users\hp\Downloads\CRAFT-pytorch\updated_extracted_text.xlsx'
data.to_excel(output_file, index=False)

print(f"Updated extracted text saved to {output_file}")